In [33]:
import os
from biopandas.pdb import PandasPdb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from rdkit.Chem import AllChem
from scipy.spatial.transform import Rotation as R
# Function to create a 20x20x20 grid with 20 features
#inout pdb file
#replace with inputpdb file location.
input_pdb = r"C:\Users\skosaraju\OneDrive - Cal Poly Pomona\Western University\TM-ligand\TM-ligand\6kqi-9GL.pdb"
pdb_df =  PandasPdb().read_pdb(input_pdb)

#this function separates protein name
ligind_name = input_pdb.split('\\')[-1].split('-')[-1][0:3]
protein_name = input_pdb.split('\\')[-1].split('-')[0]
pdb_df.df.keys()
print(ligind_name,protein_name)
output_path = r"C:\Users\skosaraju\OneDrive - Cal Poly Pomona\Western University\%s"%ligind_name
os.makedirs(output_path)
# pdb = pdbreader.read_pdb(r"C:\Users\skosaraju\OneDrive - Cal Poly Pomona\Western University\TM-ligand\TM-ligand\4xdl-40D.pdb")

9GL 6kqi


In [34]:
def reading_pdb(pdb_df,ligind_name):
    atom_df_a_atom = pdb_df.df['ATOM']
    atom_df_a_hetatm = pdb_df.df['HETATM']
    # print(atom_df_a_hetatm) 
    atom_anisou = pdb_df.df['ANISOU']
    
    atom_df_a_atom['residue_number']
    atom  = atom_df_a_hetatm[atom_df_a_hetatm['residue_name']==ligind_name]
    x = list(set(atom['residue_number']))
    return atom,atom_df_a_atom,x
def grid_list(atom1):
    grid_list1 = []
    for i in range(len(atom1)):
        grid_list1.append((atom1.iloc[i]['x_coord'],atom1.iloc[i]['y_coord'],atom1.iloc[i]['z_coord']))
    return grid_list1
def centeroid(grid_list1):
    points_array = np.array(grid_list1)


    centroid = np.mean(points_array, axis=0)
    return centroid
def filtering_proteins(atom_df_a_atom,centroid,A):

    condition = (centroid[0]-(A/2)< atom_df_a_atom['x_coord'])  &(atom_df_a_atom['x_coord']<(A/2)+centroid[0]) & (centroid[1]-(A/2)< atom_df_a_atom['y_coord']) &( atom_df_a_atom['y_coord']<centroid[1]+(A/2))  & (centroid[2]-(A/2)< atom_df_a_atom['z_coord']) &(atom_df_a_atom['z_coord']<centroid[2]+(A/2))
    filtered_protein = atom_df_a_atom[condition]
    return filtered_protein
amino_acid_smiles = {
    'ALA': 'CC(C(=O)O)N',   # Alanine
    'GLY': 'NCC(=O)O',      # Glycine
    'SER': 'C(C(C(=O)O)N)O',# Serine
    'LYS': 'CC(C(C(=O)O)N)CCCCN',  # Lysine
    'TYR': 'CC(C(C(=O)O)N)c1ccc(O)cc1',  # Tyrosine
    'THR': 'C[C@H](O)[C@H](N)C(=O)O',  # Threonine
    'ILE': 'CC[C@H](C)[C@H](N)C(=O)O',  # Isoleucine
    'PHE': 'C1=CC=C(C=C1)CC(C(=O)O)N',  # Phenylalanine
    'LEU': 'CC(C)C[C@H](N)C(=O)O',  # Leucine
    'CYS': 'C(C(C(=O)O)N)S',  # Cysteine
    'VAL': 'CC(C)C(C(=O)O)N',  # Valine
    'PRO': 'C1CC(NC1)C(=O)O',  # Proline
    'TRP': 'C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N'  # Tryptophan
}
def protein_dataframe_to_smiles(df, amino_acid_smiles):
    smiles_list = []

    # Loop over each residue in the DataFrame
    for index, row in df.iterrows():
        residue = row['residue_name']
        
        # Get the SMILES for the residue from the mapping
        if residue in amino_acid_smiles:
            smiles_list.append(amino_acid_smiles[residue])
        else:
            print(f"Unknown amino acid: {residue}")

    # Concatenate SMILES for the peptide chain (using '.' to separate the amino acids)
    # In a real scenario, you would want to handle the peptide bonds between amino acids
    combined_smiles = ".".join(smiles_list)
    return combined_smiles
def smiles_to_smarts(smiles_string):
    # Convert the SMILES string to an RDKit molecule
    mol = Chem.MolFromSmiles(smiles_string)
    
    if mol:
        # Convert the molecule to a SMARTS string
        smarts = Chem.MolToSmarts(mol)
        return smarts
    else:
        print("Failed to create molecule from SMILES.")
        return None
def save_smiles_to_txt(smiles, output_file):
    with open(output_file, 'w') as f:
        f.write(smiles)
    print(f"SMILES string saved to {output_file}")

def visualize_smarts(smiles_string, smarts_string):
    # Convert the SMILES string to a molecule
    mol = Chem.MolFromSmiles(smiles_string)
    
    # Convert the SMARTS string to a substructure query
    smarts = Chem.MolFromSmarts(smarts_string)
    
    if mol and smarts:
        # Find substructure match (atom indices)
        match = mol.GetSubstructMatch(smarts)
        
        if match:
            print(f"Matched atoms: {match}")
            # Highlight the matched atoms in the molecule
            img = Draw.MolToImage(mol, highlightAtoms=match)
            return img
        else:
            print("No match found for the SMARTS pattern.")
            return None
    else:
        print("Failed to create molecule or SMARTS pattern.")
        return None
def visualize_smarts_in_3d(smiles_string, smarts_string,html_filename):
    # Create the molecule from SMILES
    mol = Chem.MolFromSmiles(smiles_string)
    if not mol:
        print("Failed to create molecule from SMILES.")
        return

    # Add hydrogens
    mol = Chem.AddHs(mol)
    
    # Generate 3D coordinates
    AllChem.EmbedMolecule(mol)

    # Convert SMARTS to a query molecule
    query = Chem.MolFromSmarts(smarts_string)
    if not query:
        print("Failed to create SMARTS query.")
        return

    # Find substructure matches
    matches = mol.GetSubstructMatches(query)
    highlight_atoms = [atom_id for match in matches for atom_id in match]

    # Convert the RDKit molecule to a PDB block
    pdb_block = Chem.MolToPDBBlock(mol)

    # Set up the 3Dmol view
    viewer = py3Dmol.view(width=800, height=400)
    viewer.addModel(pdb_block, 'pdb')
    viewer.setStyle({'stick': {}})  # Default style

    # Highlight the atoms that match the SMARTS pattern
    viewer.setStyle({'stick': {'color': 'red'}}, {'serial': highlight_atoms})

    # Zoom to fit the molecule in the view
    viewer.zoomTo()
    with open(html_filename, 'w') as f:
        f.write(viewer._make_html())

        print(f"Visualization saved to {html_filename}")
    # Show the viewer
    # viewer.show()
# import numpy as np
# from rdkit.Chem import AllChem
# from scipy.spatial.transform import Rotation as R
# Function to create a 20x20x20 grid with 20 features per grid point
def create_grid(size=20, resolution=1):
    grid = np.zeros((size, size, size, 20))  # 20 features per grid point
    return grid

# Function to convert SMILES to RDKit molecule
def create_molecule_from_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print("Invalid SMILES string.")
        return None
    mol = Chem.AddHs(mol)
    try:
        AllChem.EmbedMolecule(mol) 
        AllChem.UFFOptimizeMolecule(mol)  # Generate 3D coordinates
    except Exception as e:
        print(f"Error generating 3D coordinates: {e}")
        return None
    return mol
    
# Function to apply 3D rotation to atomic coordinates
def rotate_molecule(mol, rotation_matrix):
    
    conf = mol.GetConformer()
    for atom_idx in range(mol.GetNumAtoms()):
        pos = conf.GetAtomPosition(atom_idx)
        new_pos = np.dot(rotation_matrix, np.array([pos.x, pos.y, pos.z]))
        conf.SetAtomPosition(atom_idx, new_pos)
    return mol

# Generate a random rotation matrix
def generate_random_rotation_matrix():
    # Generate a random 3D rotation using Euler angles
    rotation = R.from_euler('xyz', np.random.uniform(0, 360, size=3), degrees=True)
    return rotation.as_matrix()

# Function to encode atomic features (same as before)
atom_types = {'C': 0, 'N': 1, 'O': 2, 'S': 3, 'other': 4}

def encode_atom_features(atom):
    features = np.zeros(14)
   # One-hot encoding for atom types
    atom_symbol = atom.GetSymbol()
    if atom_symbol in atom_types:
        features[atom_types[atom_symbol]] = 1
    else:
        features[atom_types['other']] = 1
    
    hybridization = atom.GetHybridization()
    if hybridization == Chem.HybridizationType.SP:
        features[5] = 1
    elif hybridization == Chem.HybridizationType.SP2:
        features[6] = 1
    elif hybridization == Chem.HybridizationType.SP3:
        features[7] = 1

    num_heavy_atoms = sum(1 for neighbor in atom.GetNeighbors() if neighbor.GetAtomicNum() > 1)
    features[8] = num_heavy_atoms
   
    # Number of bonded hetero atoms (atoms other than carbon and hydrogen)
    num_hetero_atoms = sum(1 for neighbor in atom.GetNeighbors() if neighbor.GetAtomicNum() not in {1, 6})
    features[9] = num_hetero_atoms
    features[10] = 1 if atom.GetIsAromatic() else 0
    features[11] = 1 if atom.GetFormalCharge() != 0 else 0
   
    features[12] = atom.GetFormalCharge()
    
    features[13] = 1 if atom.IsInRing() else 0
    
    # print("isring:",features[13])
    return features

# Function to perform one-hot encoding for residue types
def encode_residue_type(residue):
    features = np.zeros(6)
    if residue in ['D', 'E']:
        features[0] = 1
    elif residue in ['K', 'R']:
        features[1] = 1
    elif residue == 'H':
        features[2] = 1
    elif residue == 'C':
        features[3] = 1
    elif residue in ['N', 'Q', 'W', 'Y', 'S', 'T']:
        features[4] = 1
    else:
        features[5] = 1
    return features

# Map atoms to the grid based on their 3D coordinates
def map_atoms_to_grid(mol, grid, grid_center, grid_size=20, resolution=1):
    conf = mol.GetConformer()
    
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        grid_coord = (np.array([pos.x, pos.y, pos.z]) - grid_center) // resolution
        grid_coord = grid_coord.astype(int)
        
        if np.all(grid_coord >= 0) and np.all(grid_coord < grid_size):
            atom_features = encode_atom_features(atom)
            residue = atom.GetPDBResidueInfo().GetResidueName() if atom.GetPDBResidueInfo() else 'Other'
            residue_features = encode_residue_type(residue)
            combined_features = np.concatenate((atom_features, residue_features))
            grid[tuple(grid_coord)] = combined_features

    return grid

# Main function to generate multiple rotated grids
def generate_rotated_grids(smiles, grid_center, num_rotations=20, grid_size=20, resolution=1):
    mol = create_molecule_from_smiles(smiles)
    
    if mol is None:
        return None
    
    grids = []
    
    for i in range(num_rotations):
        # Create a new grid
        grid = create_grid(size=grid_size, resolution=resolution)
        
        # Generate a random rotation matrix
        rotation_matrix = generate_random_rotation_matrix()
        
        # Rotate the molecule
        rotated_mol = rotate_molecule(mol, rotation_matrix)
        
        # Map rotated atoms to the grid
        grid = map_atoms_to_grid(rotated_mol, grid, grid_center, grid_size, resolution)
        
        # Store the rotated grid
        grids.append(grid)
    
    return grids
def saving_features(rotated_grids,output_path,protein_name_):
    featureGrid = output_path+'/'+protein_name_+'/'+"featureGrid"
    out =  output_path+'/'+protein_name_
    os.makedirs(out)
    os.makedirs(featureGrid)
    featureCSV = output_path+'/'+protein_name_+'/'+"featureCSV"
    os.makedirs(featureCSV)
    # Save each grid
    for idx, grid in enumerate(rotated_grids):
        np.save(f'{featureGrid}/rotated_grid_{idx}.npy', grid)
        flattened_grid = grid.reshape(-1, grid.shape[-1])  # Flatten to 2D
        np.savetxt(f'{featureCSV}/rotated_grid_{idx}.csv', flattened_grid, delimiter=',')
        print(f"Saved rotated grid {idx} successfully.")
    return




In [35]:
atom,atom_df_a_atom,x = reading_pdb(pdb_df,ligind_name)
A  = 8
# print(x)
for i in range(len(x)):
    # print(x[i])
    atom_ = atom[atom['residue_number']==x[i]]
    # print(atom_)
    grid_list_ = grid_list(atom_)
    # print(grid_list_)

    centroid_ = centeroid(grid_list_)
    # print(centroid_)
    filtering_proteins_ = filtering_proteins(atom_df_a_atom,centroid_,A)
    # print(filtering_proteins_)
    smiles = protein_dataframe_to_smiles(filtering_proteins_, amino_acid_smiles)
    if smiles:
        print(f"smiles: {smarts_string}")
    output_file = r'%s/%s%s_smile_string.txt'%(output_path,protein_name,x[i])
    print(smiles)
    smarts_string = smiles_to_smarts(smiles)
    # smiles = protein_dataframe_to_smiles(filtered_protein, amino_acid_smiles)
    if smarts_string:
        print(f"smiles: {smarts_string}")
    output_file = r'%s/%s%s_smart_string.txt'%(output_path,protein_name,x[i])
    img = visualize_smarts(smiles, smarts_string)
    if img:
        img.save("%s/%s%s_smart2e.png"%(output_path,protein_name,x[i]))
        img.show()  #
    output_html = "%s/%s%s_smart_.html"%(output_path,protein_name,x[i])
    print(output_html)
    visualize_smarts_in_3d(smiles, smarts_string,output_html)
    smiles = smiles  # Example: glucose molecule
    grid_center = np.array([0, 0, 0])  # Grid center at origin
    
    # Generate rotated grids (20 rotations)
    rotated_grids = generate_rotated_grids(smiles, grid_center, num_rotations=20)
    protein_name_ = protein_name+'%s'%x[i]
    saving_features(rotated_grids,output_path,protein_name_)

    

smiles: [#6]1:[#6]:[#6]:[#6](:[#6]:[#6]:1)-[#6]-[#6](-[#6](=[#8])-[#8])-[#7].[#6]1:[#6]:[#6]:[#6](:[#6]:[#6]:1)-[#6]-[#6](-[#6](=[#8])-[#8])-[#7].[#6]1:[#6]:[#6]:[#6](:[#6]:[#6]:1)-[#6]-[#6](-[#6](=[#8])-[#8])-[#7].[#6]1:[#6]:[#6]:[#6](:[#6]:[#6]:1)-[#6]-[#6](-[#6](=[#8])-[#8])-[#7]
C(C(C(=O)O)N)S.C(C(C(=O)O)N)S.C(C(C(=O)O)N)S.C(C(C(=O)O)N)S.C(C(C(=O)O)N)S.C(C(C(=O)O)N)S.C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N.C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N.C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N.C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N.C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N
smiles: [#6](-[#6](-[#6](=[#8])-[#8])-[#7])-[#16].[#6](-[#6](-[#6](=[#8])-[#8])-[#7])-[#16].[#6](-[#6](-[#6](=[#8])-[#8])-[#7])-[#16].[#6](-[#6](-[#6](=[#8])-[#8])-[#7])-[#16].[#6](-[#6](-[#6](=[#8])-[#8])-[#7])-[#16].[#6](-[#6](-[#6](=[#8])-[#8])-[#7])-[#16].[#6]1:[#6]:[#6]:[#6]2:[#6](:[#6]:1):[#6](:[#6]:[#7H]:2)-[#6]-[#6](-[#6](=[#8])-[#8])-[#7].[#6]1:[#6]:[#6]:[#6]2:[#6](:[#6]:1):[#6](:[#6]:[#7H]:2)-[#6]-[#6](-[#6](=[#8])-[#8])-[#7].[#6]1:[#6]:[#6]:[#6]2

In [19]:
atom_df_a_atom

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,LYS,,A,73,,...,-28.094,142.686,30.504,1.0,159.97,,,N,NaN,1040
1,ATOM,2,,CA,,LYS,,A,73,,...,-28.439,142.044,31.774,1.0,162.97,,,C,NaN,1042
2,ATOM,3,,C,,LYS,,A,73,,...,-27.249,141.274,32.341,1.0,166.15,,,C,NaN,1044
3,ATOM,4,,O,,LYS,,A,73,,...,-26.426,140.799,31.561,1.0,162.76,,,O,NaN,1046
4,ATOM,5,,CB,,LYS,,A,73,,...,-29.629,141.095,31.584,1.0,168.27,,,C,NaN,1048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7469,ATOM,7473,,N,,LEU,,D,330,,...,67.262,65.469,35.522,1.0,195.47,,,N,NaN,15981
7470,ATOM,7474,,CA,,LEU,,D,330,,...,66.760,65.686,36.882,1.0,219.98,,,C,NaN,15983
7471,ATOM,7475,,C,,LEU,,D,330,,...,65.578,64.765,37.180,1.0,239.13,,,C,NaN,15985
7472,ATOM,7476,,O,,LEU,,D,330,,...,65.367,64.378,38.325,1.0,201.20,,,O,NaN,15987
